In [ ]:
import video_data_classes as vid
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from skimage import io
from importlib import reload
from IPython.core.debugger import set_trace
from torch.utils.data import Dataset, DataLoader

In [32]:
class SuperResNetwork(nn.Module):
    def __init__(self):
        super(SuperResNetwork, self).__init__()

        self.input_layer = nn.Conv2d(3, 8, (3, 3))
        self.output_layer = nn.ConvTranspose2d(8, 3, (3, 3))
    
    def forward(self, x):
        first_input = x[0, :, :, :, :]
        first_input = first_input.double()
        sequence_length = x.shape[0]
        set_trace()
        y = self.input_layer(torch.cat([first_input, torch.zeros_like(first_input)], dim=2)) # concatenate the first input with zeros in the channel dimension
        y = F.ReLU(y)
        y = self.output_layer(y)
        output = torch.reshape(y, (3, 256, 256))
        
        for i in range(1, sequence_length):
            y = self.input_layer(torch.cat([x[i:i+1, :, :, :, :], output[i - 1:i, :, :, :, :]], dim=2))
            y = F.ReLU(y)
            y = self.output_layer(y)
            output = torch.cat([output, torch.reshape(y, (3, 256, 256))], dim=0)
            
        return output

In [33]:
network = SuperResNetwork()
criterion = nn.MSELoss()
optimizer = optim.Rprop(network.parameters())

video_directory = r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs'
preprocess = transforms.Compose([vid.LightenImageTransformer(),
                                vid.MinMaxTransformer()])
dataset = vid.VideoDataset(video_directory, 10, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
for i, data in enumerate(dataloader):
    train_x = torch.stack(data['X']).permute(0, 1, 4, 3, 2) #seq, batch, c, h, w
    train_y = torch.stack(data['Y']).permute(0, 1, 4, 3, 2)
    
    optimizer.zero_grad()
    
    outputs = network(train_x)
    loss = criterion(outputs, train_y)
    loss.backward()
    optimizer.step()

In [ ]:
data = vid.VideoDataset(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs', 10)

In [ ]:
type(dataset[0])